In [1]:
import numpy as np
import os
import pandas as pd
import torch
import albumentations as A
import albumentations.pytorch
import cv2
import sys
import random
import csv
import json
import matplotlib.pyplot as plt

sys.path.append('../')
from visualize import visualize
from utils import *

%matplotlib inline

In [2]:
df = pd.read_csv('../saved/df.csv')
df['label'] = df.label.apply(lambda x : CLASS_MAPPER[str(x)])
df = drop_wrong(df)
df.reset_index(drop=True, inplace=True)
df.shape

(22662, 14)

In [3]:
df.tail()

,file_name,task,label,xmin,ymin,w,h,occluded,des,cell_type,bbox,area,ratio,ID
22657,patch_images/2021.01.08/LBC131-20210105(1)/LBC...,[ASCUS] LBC131,ASC-US,755,755,179,79,0,NaN,Atypical squamous cells of undetermined signif...,"[755, 755, 179, 79]",118,1.505265,17805
22658,patch_images/2021.01.08/LBC131-20210105(1)/LBC...,[ASCUS] LBC131,ASC-US,498,1087,147,110,0,NaN,Atypical squamous cells of undetermined signif...,"[498, 1087, 147, 110]",127,1.156012,17806
22659,patch_images/2021.01.08/LBC131-20210105(1)/LBC...,[ASCUS] LBC131,ASC-US,461,1222,118,150,0,NaN,Atypical squamous cells of undetermined signif...,"[461, 1222, 118, 150]",133,0.886942,17806
22660,patch_images/2021.01.08/LBC131-20210105(1)/LBC...,[ASCUS] LBC131,ASC-US,1311,356,147,153,0,NaN,Atypical squamous cells of undetermined signif...,"[1311, 356, 147, 153]",149,0.980196,17807
22661,patch_images/2021.01.08/LBC131-20210105(1)/LBC...,[ASCUS] LBC131,ASC-US,203,372,139,87,0,NaN,Atypical squamous cells of undetermined signif...,"[203, 372, 139, 87]",109,1.264002,17808


In [4]:
columns = 'label'
all_size = len(df)
ascus_df = df[df[columns]=='ASC-US']
asch_df = df[df[columns]=='ASC-H']
neg_df = df[df[columns]=='Negative']
hsil_df = df[df[columns]=='HSIL']
lsil_df = df[df[columns]=='LSIL']
carcinoma_df = df[df[columns]=='Carcinoma']
sum_partial = len(ascus_df) + len(asch_df) + len(neg_df) + len(hsil_df) + len(lsil_df) + len(carcinoma_df)
print(all_size, sum_partial)

22662 22662


In [5]:
def split_by_group_ratio(tdf, test_ratio, seed ) :
    df = tdf.copy()
    df.reset_index(drop=True, inplace=True)
    TRAIN_TEST_SPLIT_PERC = 1- test_ratio
    uniques = tdf["task"].unique()
    sep = int(len(uniques) * TRAIN_TEST_SPLIT_PERC)

    best_ratio = 0
    best_unique = []
    best_train_df = []
    best_test_df = []
    df = tdf.copy()
    for i in range(40) :
        random.shuffle(uniques)
        
        # df = df.sample(frac=1).reset_index(drop=True) #For shuffling your data
          
        train_ids, test_ids = uniques[:sep], uniques[sep:]
        train_df, test_df = df[df.task.isin(train_ids)], df[df.task.isin(test_ids)]
        ratio = len(test_df)/(len(test_df) + len(train_df))
        if ratio > best_ratio and ratio < test_ratio :
            best_ratio = ratio
            best_unique.append(uniques)
            best_train_df.append(train_df)
            best_test_df.append(test_df)

    print(len(best_test_df[-1])/(len(best_test_df[-1]) + len(best_train_df[-1])))
    
    #return best_train_df[-1].index, best_test_df[-1].index
    return best_train_df[-1], best_test_df[-1]

In [6]:
'''
train_li = []
test_li = []
ratio = 0.25
seed = 0

# for tdf in list([ascus_df, asch_df, neg_df, hsil_df, lsil_df]) :
for tdf in list([ascus_df]) :
    tr_inds, te_inds = split_by_group_ratio(tdf, ratio, seed)
    temp_tr = tdf.iloc[tr_inds]
    temp_te = tdf.iloc[te_inds]
    train_li.append(temp_tr)
    test_li.append(temp_te)
train_df = pd.concat(train_li)
test_df = pd.concat(test_li)
'''

'\ntrain_li = []\ntest_li = []\nratio = 0.25\nseed = 0\n\n# for tdf in list([ascus_df, asch_df, neg_df, hsil_df, lsil_df]) :\nfor tdf in list([ascus_df]) :\n    tr_inds, te_inds = split_by_group_ratio(tdf, ratio, seed)\n    temp_tr = tdf.iloc[tr_inds]\n    temp_te = tdf.iloc[te_inds]\n    train_li.append(temp_tr)\n    test_li.append(temp_te)\ntrain_df = pd.concat(train_li)\ntest_df = pd.concat(test_li)\n'

In [7]:
print(len(set(ascus_df.task))+len(set(asch_df.task))+len(set(neg_df.task))+len(set(hsil_df.task))+len(set(lsil_df.task)))

584


In [19]:
# tr_df.task.unique()

In [17]:
tr_df.label.unique()

array(['LSIL'], dtype=object)

In [8]:
train_li = []
test_li = []
ratio = 0.25
seed = 0

for tdf in list([ascus_df, asch_df, neg_df, hsil_df, lsil_df]) :
#for tdf in list([ascus_df, asch_df]) :
    tr_df, te_df = split_by_group_ratio(tdf, ratio, seed)
    tr_df.reset_index(drop=True, inplace=True)
    te_df.reset_index(drop=True, inplace=True)
    train_li.append(tr_df)
    test_li.append(te_df)
    print(len(set(tr_df.task) - set(te_df.task)))
    print(len(set(tr_df.task)))
    print(len(set(te_df.task)))  
    print(tr_df.shape)
train_df = pd.concat(train_li, ignore_index=True)
test_df = pd.concat(test_li, ignore_index=True)

0.24771637876458352
225
225
76
(8318, 14)
0.23183673469387756
48
48
16
(941, 14)
0.24403619413216343
13
13
5
(2757, 14)
0.22987721691678034
42
42
15
(1129, 14)
0.24714176829268292
108
108
36
(3951, 14)


In [9]:
train_df.shape


(17096, 14)

In [10]:
train_df.drop_duplicates().shape

(17096, 14)

In [11]:
len(set(train_df.task)) + len(set(test_df.task))

409

In [12]:
print(len(set(train_df.task) - set(test_df.task)))
print(len(set(train_df.task)))
print(len(set(test_df.task)))

202
283
126


In [13]:
list(set(train_df.task))

['[ASCUS] LBC305',
 '[ASCUS] LBC77',
 '[ASCUS] LBC182',
 '[AS6] LBC417',
 '[ASCUS] LBC68',
 '[ASCUS] LBC39',
 '[ASCUS] LBC570',
 'LBC1172',
 'LBC890',
 '[LSIL] LBC599',
 '[AS6] LBC290',
 '[ASCUS] LBC69',
 '[ASCUS] LBC140',
 '[LSIL] LBC199',
 '[ASCUS] LBC137',
 '[ASCUS] LBC183',
 '[AS6] LBC426',
 '[ASCUS] LBC572',
 'LBC2167',
 '[AS6] LBC425',
 '[AS6] LBC409',
 '[AS6] LBC411',
 '[AS6] LBC344',
 'LBC2042',
 '[ASCUS] LBC71',
 'LBC1018',
 '[LSIL] LBC85',
 '[ASCUS] LBC231',
 '[ASC-H] LBC143',
 'LBC1568',
 '[ASC-H] LBC605',
 '[ASCUS] LBC301',
 'LBC1021',
 '[ASCUS] LBC574',
 '[ASCUS] LBC242',
 '[ASCUS] LBC581',
 'LBC747',
 '[LS6] LBC248',
 'LBC1857',
 '[ASCUS] LBC575',
 '[LSIL] LBC428',
 'LBC52',
 '[ASC-H] LBC201',
 '[LSIL] LBC310',
 '[AS6] LBC298',
 '[AS6] LBC285',
 '[ASCUS] LBC192',
 '[LSIL] LBC84',
 'LBC1945',
 '[ASCUS] LBC33',
 '[ASCUS] LBC78',
 '[ASCUS] LBC195',
 'LBC1567',
 '[HSIL] LBC608',
 '[ASCUS] LBC186',
 '[ASCUS] LBC196',
 '[ASCUS] LBC40',
 'LBC1792',
 'LBC1349',
 '[ASCUS] LBC79',
